In [1]:
import pandas as pd
import numpy as np
import requests
import json
import re, textwrap
import pandas as pd
import numpy as np
import argparse

In [2]:
!pip install TwitterAPI

In [4]:
#NOTE: Must have TwitterAPI Installed
from TwitterAPI import TwitterAPI
from TwitterAPI import TwitterPager

In [5]:
api_key="LhCSedmn8rIMMipticrS4g350"
api_secret_key="y4Ws7XPuV9cmsOS3aId4LPIS5kMiA7nFzldBblOZo5lxfmC032"
bearer="AAAAAAAAAAAAAAAAAAAAAB91GAEAAAAA24kbznYamgCYk%2FRMiXyttUXILIU%3DIazdeos2q8xEOP4GkXAaK6gX37u7lx0Gq6hrg9MT1z6mtAMDgO"
AccessToken="1285002674597851137-IXKri9EFCqh9EozdKZqTqKL7BSXDWI"
AccessTokenSecret="BjJM0o0X1hZ8hjUwYDUfeSzWrSPJjgr1D5wXfGWQeZh8j"

In [6]:
# Setup Twitter API Client
api = TwitterAPI(api_key, 
                 api_secret_key,
                 AccessToken,
                 AccessTokenSecret)

In [7]:
def remove_non_ascii_2(text):
    """
    helper function to remove non-ascii character like emojis and things
    """
    return re.sub(r'[^\x00-\x7F]+', "'", text)

def extract_tweet_info(item, count, term):
    """
    Extract columns for each tweet
    """
    # Variables
    time = item['created_at'] # might need to use python to format
    tweet = remove_non_ascii_2(item['text']) #tweet text
    search_term = term # search term
    language = item['lang'] # language of the tweet
    retweet_count = item['retweet_count'] # retweet count
    user_id = item['user']['id'] # user id of the author
    is_verified_user = item['user']['verified'] # is the user verified (binary)
    user_friend_count = item['user']['friends_count'] # number of author's friends lol
    user_follower_count = item['user']['followers_count'] # number of author's followers
    user_location = item['user']['location']
    
    # clean up hashtags to only extract text
    hashtags = item['entities']['hashtags']
    hashtags_result = []
    for x in hashtags:
        hashtags_result.append(x['text'])

    #Add Row to Data Frame
    new_row = {"created_at":time, 
               "tweet":tweet, 
               "search_term": search_term, 
               "hashtags":str(hashtags_result).strip('[]'), 
               "language":language,
               "retweet_count":retweet_count,
               "user_id":user_id,
               "is_verified_user":is_verified_user,
               "user_follower_count":user_friend_count,
               "user_friend_count":user_follower_count,
               "user_location":user_location
              }
    
    return new_row

https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

Returns tweets by users located within a given radius of the given latitude/longitude. The location is preferentially taking from the Geotagging API, but will fall back to their Twitter profile.

In [52]:
# TODO: Update these with the right parameters
limit = 500
HASHTAGS = '#nuggets'
#location_box = '29,-95'

# create a Twitter paging client, will request one page at a time
r = TwitterPager(api, 
                 'search/tweets', 
                 {'q': HASHTAGS})

In [53]:
#Setup Initial Empty Data Frame
header = [
          "created_at", 
          "tweet", 
          "search_term", 
          "hashtags", 
          "language",
          "retweet_count",
          "user_id",
          "is_verified_user",
          "user_follower_count",
          "user_friend_count",
          "user_location",
         ]
index = np.arange(0)
df = pd.DataFrame(columns=header, index = index)

In [54]:
count = 0 # counter
print("requested tweets for hashtag is limited to {} tweets".format(limit))
for item in r.get_iterator(wait=6):
    if 'text' in item:
        if count <= limit:
            if count % 10 == 0:
                print("collecting tweet {} of {}...".format(count, limit))
            count += 1
            #Extract Tweet Info
            new_row = extract_tweet_info(item, count, SEARCH_TERM)
            df = df.append(new_row, ignore_index=True)

        else:
            print("requested tweet limit reached...")
            print("ending query for hashtag...")
            break

    elif 'message' in item and item['code'] == 88:
        print('SUSPEND, RATE LIMIT EXCEEDED: %s' % item['message'])
        break

requested tweets for hashtag is limited to 500 tweets
collecting tweet 0 of 500...
collecting tweet 10 of 500...
collecting tweet 20 of 500...
collecting tweet 30 of 500...
collecting tweet 40 of 500...
collecting tweet 50 of 500...
collecting tweet 60 of 500...
collecting tweet 70 of 500...
collecting tweet 80 of 500...
collecting tweet 90 of 500...
collecting tweet 100 of 500...
collecting tweet 110 of 500...
collecting tweet 120 of 500...
collecting tweet 130 of 500...
collecting tweet 140 of 500...
collecting tweet 150 of 500...
collecting tweet 160 of 500...
collecting tweet 170 of 500...
collecting tweet 180 of 500...
collecting tweet 190 of 500...
collecting tweet 200 of 500...
collecting tweet 210 of 500...
collecting tweet 220 of 500...
collecting tweet 230 of 500...
collecting tweet 240 of 500...
collecting tweet 250 of 500...
collecting tweet 260 of 500...
collecting tweet 270 of 500...
collecting tweet 280 of 500...
collecting tweet 290 of 500...
collecting tweet 300 of 500

In [51]:
df.head()

,created_at,tweet,search_term,hashtags,language,retweet_count,user_id,is_verified_user,user_follower_count,user_friend_count,user_location
0,Fri Sep 25 04:21:41 +0000 2020,#Nuggets lead the series 1-3,nuggets,'Nuggets',en,0,57270423,False,1182,311,
1,Fri Sep 25 04:21:30 +0000 2020,It's time!! #Nuggets!! https://t.co/WqRz3O0AKW,nuggets,'Nuggets',en,0,60448531,False,1717,1002,
2,Fri Sep 25 04:21:30 +0000 2020,This is comical. They actually asked the leagu...,nuggets,'Nuggets',en,0,56868379,False,76,103,"Centennial, CO"
3,Fri Sep 25 04:21:29 +0000 2020,RT @ESPNDatos: #LeBronJames lidera a los #Lake...,nuggets,"'LeBronJames', 'Lakers', 'Nuggets'",es,16,1228383504330895361,False,121,142,Choluteca
4,Fri Sep 25 04:21:15 +0000 2020,RT @1043TheFan: RAPID REACTION | #Nuggets down...,nuggets,'Nuggets',en,5,2587240453,False,1048,1045,"Colorado Springs, CO"


In [55]:
# save the dataframe to csv
print(df.shape)
df.to_csv("nuggets2.csv")

(501, 11)
